In [56]:
# TODO: Move to setup file
import bs4 as bs
import requests
import numpy as np

# Todo: trim
import keras
from keras import backend as K
from keras.utils.data_utils import get_file
from keras.utils import np_utils
from keras.utils.np_utils import to_categorical
from keras.models import Sequential, Model
from keras.layers import Input, Embedding, Reshape, merge, LSTM, Bidirectional
from keras.layers import TimeDistributed, Activation, SimpleRNN, GRU
from keras.layers.core import Flatten, Dense, Dropout, Lambda
from keras.regularizers import l2, activity_l2, l1, activity_l1
from keras.layers.normalization import BatchNormalization
from keras.optimizers import SGD, RMSprop, Adam
from keras.utils.layer_utils import layer_from_config
from keras.metrics import categorical_crossentropy, categorical_accuracy
from keras.layers.convolutional import *
from keras.preprocessing import image, sequence
from keras.preprocessing.text import Tokenizer

# Get text from URL

In [ ]:
response = requests.get('http://www.hplovecraft.com/writings/texts/fiction/cc.aspx')
soup = bs.BeautifulSoup(response.text, 'html5lib')
text = soup.findAll(align="justify")[0]

In [ ]:
print(text.find(align="justify").get_text(strip=True))

In [ ]:
with open ("The_Call_of_Cthulhu.txt", 'wb') as f:
    f.write(text.find(align="justify").get_text(strip=True).encode("utf8"))

In [ ]:
print(len(text.find(align="justify").get_text(strip=True)))

# Read and format local text doc

In [2]:
training_text = open('../data/text/The_Call_of_Cthulhu.txt').read()

In [3]:
chars = sorted(list(set(training_text)))
vocab_size = len(chars)+1
print('total chars:', vocab_size)
chars.insert(0, "\0")
''.join(chars[1:-6])

total chars: 85


'\n !&(),-.0123456789:;?ABCDEFGHIJKLMNOPQRSTUVWYZabcdefghijklmnopqrstuvwxyz\xa0°Åéë'

In [4]:
# Create dict to lookup character index but character
char_indices = dict((c, i) for i, c in enumerate(chars))
# Create dict to lookup index by charater
indices_char = dict((i, c) for i, c in enumerate(chars))
print(char_indices, indices_char)

{'8': 18, 'T': 42, '5': 15, 'i': 56, '°': 75, 'I': 31, 'x': 71, '3': 13, 'é': 77, 'H': 30, 'P': 38, 'l': 59, 'O': 37, 'z': 73, '!': 3, 'q': 64, '\n': 1, 'Å': 76, '0': 10, '–': 79, 'f': 53, 'h': 55, 'M': 35, 'y': 72, 'o': 62, ' ': 2, 'K': 33, 'N': 36, 't': 67, 'j': 57, 'Y': 46, 'C': 25, 'U': 43, ',': 7, 'e': 52, 'ë': 78, '’': 81, '1': 11, 'D': 26, 'G': 29, 'J': 32, 's': 66, 'r': 65, 'c': 50, '6': 16, 'm': 60, '(': 5, 'n': 61, 'L': 34, '.': 9, 'v': 69, 'S': 41, '9': 19, 'g': 54, ';': 21, ')': 6, 'd': 51, 'R': 40, '&': 4, 'B': 24, ':': 20, '2': 12, '?': 22, 'w': 70, '—': 80, 'b': 49, 'p': 63, 'k': 58, 'u': 68, '-': 8, '′': 84, 'a': 48, '”': 83, 'F': 28, '\xa0': 74, 'A': 23, '“': 82, 'Z': 47, '4': 14, '7': 17, 'V': 44, 'E': 27, 'Q': 39, 'W': 45, '\x00': 0} {0: '\x00', 1: '\n', 2: ' ', 3: '!', 4: '&', 5: '(', 6: ')', 7: ',', 8: '-', 9: '.', 10: '0', 11: '1', 12: '2', 13: '3', 14: '4', 15: '5', 16: '6', 17: '7', 18: '8', 19: '9', 20: ':', 21: ';', 22: '?', 23: 'A', 24: 'B', 25: 'C', 26: 'D',

In [5]:
#text converted to indexes
idx = [char_indices[c] for c in training_text]

In [6]:
# Test conversion
''.join(indices_char[i] for i in idx[:100])

'(Found Among the Papers of the LateFrancis Wayland Thurston, of Boston)“Of such great powers or bein'

# Keras RNN
## Predict 9th Char based on last 8

In [7]:
# Set Keras constants
n_hidden, n_fac, cs, vocab_size = (256, 42, 8, 86)

In [8]:
# Create input array of converted text, sets of 8, for input layer
c_in_dat = [[idx[i+n] for i in range(0, len(idx)-1-cs, cs)]
           for n in range(cs)]

In [15]:
# Create array of expected out (9th character)
c_out_dat = [idx[i+cs] for i in range(0, len(idx)-1-cs,cs)]

In [21]:
xs = [np.stack(c[:-2]) for c in c_in_dat]
len(xs), xs[0].shape

(8, (8690,))

In [19]:
y = np.stack(c_out_dat[:-2])
len(y), y.shape

(8690, (8690,))

In [10]:
'''Sequential - build a linear NN of layers where layer feeds
next layer. 
TODO: Look up EMbedding and Dense'''
model = Sequential ([
    Embedding(vocab_size, n_fac, input_length=cs),
    SimpleRNN(n_hidden, activation='relu', inner_init='identity'),
    Dense(vocab_size, activation='softmax')
])

In [11]:
# TODO: look up what this means
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
embedding_1 (Embedding)          (None, 8, 42)         3612        embedding_input_1[0][0]          
____________________________________________________________________________________________________
simplernn_1 (SimpleRNN)          (None, 256)           76544       embedding_1[0][0]                
____________________________________________________________________________________________________
dense_1 (Dense)                  (None, 86)            22102       simplernn_1[0][0]                
Total params: 102,258
Trainable params: 102,258
Non-trainable params: 0
____________________________________________________________________________________________________


In [12]:
model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam())
# TODO: Lookup other options

In [ ]:
# Warning: Takes a while. Don't kill page
model.fit(np.stack(xs, 1), y, batch_size=128, nb_epoch=8)

Epoch 1/8
8690/8690 [==============================] - 3s - loss: 0.7253     

In [28]:
# Get next prediction
def get_next_keras(seed):
    idxs = [char_indices[c] for c in seed]
    arrs = np.array(idxs)[np.newaxis,:]
    prediction = model.predict(arrs)[0]
    return chars[np.argmax(prediction)]

In [29]:
get_next_keras('the one ')

'h'

In [53]:
# Get a lot of predictions
def get_next_x_keras(seed, predict_length):
    result = '' + seed
    for i in range(predict_length):
        result += get_next_keras(result[-8:])
    
    return result

In [55]:
get_next_x_keras('the one ', 1000)

'the one he had frequmyt and so ble noced in the dumzront encitel. All this, a due fortor tith of tho mess and deand. Thery had aven mens res mens..OTTempmicel a pripiinty, aang hithoug themy phastior watt the pabest, ap int arone hald exppetsicy and derbans hoo houdr the mase. Thers whish and the great frimttor I phocit—ppiesse the as mentian s. whuting the soming mat was they chare burd hide blowed on Masced bot heee this was at hereath to gele the subect, a sgoungs and dreamy sowougs ad hestrate sounced on a suont wilt soumes in seversce whilh fad the redolo shpappee to bay thowe then a mistof drot of mont be rate I the cater wurh\nfiindinn of forbikn soof shimppres\nsovens mond mame a too thit wisco will a meoun farlt hay on Mar hadsers to sheal the sound\nfrom of the which was a curt of the which was a curt of the which was a curt of the which was a curt of the which was a curt of the which was a curt of the which was a curt of the which was a curt of the which was a curt of the w

# Sequence

In [57]:
'''c_in_dat = [[idx[i+n] for i in range(0, len(idx)-1-cs, cd)]
                for n in range(cs)]'''
c_out_dat = [[idx[i+n] for i in range(1, len(idx)-cs, cs)]
            for n in range(cs)]

In [58]:
ys = [np.stack(c[:-2]) for c in c_out_dat]

In [59]:
dense_in = Dense(n_hidden, activation='relu')
dense_hidden = Dense(n_hidden, activation='relu', init='identity')
dense_out = Dense(vocab_size, activation='softmax', name='output')

In [60]:
inp1 = Input(shape=(n_fac,), name='zeros')
hidden = dense_in(inp1)

In [61]:
def embedding_input(name, n_in, n_out):
    inp = Input(shape=(1,), dtype='int64', name=name+'_in')
    emb = Embedding(n_in, n_out, input_length=1, name=name+'_emb')(inp)
    return inp, Flatten()(emb)

In [62]:
c_ins = [embedding_input('c'+str(n), vocab_size, n_fac) for n in range(cs)]

In [63]:
outs = []

for i in range(cs):
    c_dense = dense_in(c_ins[i][1])
    hidden = dense_hidden(hidden)
    hidden = merge([c_dense, hidden], mode='sum')
    outs.append(dense_out(hidden))

In [64]:
model = Model([inp1] + [c[0] for c in c_ins], outs)

In [65]:
model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam())

In [66]:
zeros = np.tile(np.zeros(n_fac), (len(xs[0]),1))
zeros.shape

(8690, 42)

In [ ]:
# Warning: Takes long time ~19min on i3 1.9GHz
model.fit([zeros]+xs, ys, batch_size=64, nb_epoch=12)

Epoch 1/12
8690/8690 [==============================] - 5s - loss: 25.4111 - output_loss_1: 3.4977 - output_loss_2: 3.2993 - output_loss_3: 3.1820 - output_loss_4: 3.1387 - output_loss_5: 3.0799 - output_loss_6: 3.0421 - output_loss_7: 3.0808 - output_loss_8: 3.0905      

In [70]:
def get_next_sequence(inp):
    idxs = [char_indices[c] for c in inp]
    arrs = [np.array(i)[np.newaxis] for i in idxs]
    p = model.predict([np.zeros(n_fac)[np.newaxis,:]] + arrs)
    return chars[np.argmax(p[-1])]

In [74]:
get_next_sequence('the one ')

't'

In [72]:
def get_next_x_sequence(inp, predict_length):
    result = '' + inp
    for i in range(predict_length):
        result += get_next_sequence(result[-8:])
        
    return result

In [80]:
get_next_x_sequence('manger w', 1000)

'manger which had and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and an